***GENERATED CODE FOR interchurnclassify4 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern', 'transformation_label': 'String Indexer'}], 'feature': 'Intern', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '53', 'mean': '', 'stddev': '', 'min': 'AMRO ALHAMMEDI', 'max': 'sofia andom', 'missing': '0', 'distinct': '53'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern'}, {'feature_label': 'Intern', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intern')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternStartDate', 'transformation_label': 'String Indexer'}], 'feature': 'InternStartDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '53', 'mean': '', 'stddev': '', 'min': '1/4/2021', 'max': '9/10/2019', 'missing': '0', 'distinct': '45'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternStartDate'}, {'feature_label': 'InternStartDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternStartDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternEndDate', 'transformation_label': 'String Indexer'}], 'feature': 'InternEndDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '53', 'mean': '', 'stddev': '', 'min': '1/15/2021', 'max': 'Unkown', 'missing': '0', 'distinct': '37'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternEndDate'}, {'feature_label': 'InternEndDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternEndDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern_MM_PlacementDate', 'transformation_label': 'String Indexer'}], 'feature': 'Intern_MM_PlacementDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '53', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '4/17/2024', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern_MM_PlacementDate'}, {'feature_label': 'Intern_MM_PlacementDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intern_MM_PlacementDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternCancelReason', 'transformation_label': 'String Indexer'}], 'feature': 'InternCancelReason', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '53', 'mean': '', 'stddev': '', 'min': 'Not Quit', 'max': 'Unknown', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternCancelReason'}, {'feature_label': 'InternCancelReason', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternCancelReason')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Phase', 'transformation_label': 'String Indexer'}], 'feature': 'Phase', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '53', 'mean': '', 'stddev': '', 'min': 'Phase 1', 'max': 'Unkown', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Phase'}, {'feature_label': 'Phase', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Phase')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '53', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'Unknown', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TopDISCscore', 'transformation_label': 'String Indexer'}], 'feature': 'TopDISCscore', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '53', 'mean': '', 'stddev': '', 'min': 'Compliance', 'max': 'Unknown', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TopDISCscore'}, {'feature_label': 'TopDISCscore', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TopDISCscore')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LowDISCscore', 'transformation_label': 'String Indexer'}], 'feature': 'LowDISCscore', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '53', 'mean': '', 'stddev': '', 'min': 'Compliance', 'max': 'Unknown', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LowDISCscore'}, {'feature_label': 'LowDISCscore', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LowDISCscore')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TopMotivator', 'transformation_label': 'String Indexer'}], 'feature': 'TopMotivator', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '53', 'mean': '', 'stddev': '', 'min': 'Aesthetic', 'max': 'Utilitarian', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TopMotivator'}, {'feature_label': 'TopMotivator', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TopMotivator')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LowMotivator', 'transformation_label': 'String Indexer'}], 'feature': 'LowMotivator', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '53', 'mean': '', 'stddev': '', 'min': 'Aesthetic', 'max': 'Utilitarian', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LowMotivator'}, {'feature_label': 'LowMotivator', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LowMotivator')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TopSkill', 'transformation_label': 'String Indexer'}], 'feature': 'TopSkill', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '53', 'mean': '', 'stddev': '', 'min': 'Flexibility', 'max': 'WrittenCommunication', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TopSkill'}, {'feature_label': 'TopSkill', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TopSkill')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LowSkill', 'transformation_label': 'String Indexer'}], 'feature': 'LowSkill', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '53', 'mean': '', 'stddev': '', 'min': 'Flexibility', 'max': 'WrittenCommunication', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LowSkill'}, {'feature_label': 'LowSkill', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LowSkill')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run interchurnclassify4Hooks.ipynb
try:
	#sourcePreExecutionHook()

	aiforgemltestdata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/AIForgeMLTestData.csv', 'filename': 'AIForgeMLTestData.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(aiforgemltestdata)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run interchurnclassify4Hooks.ipynb
try:
	#transformationPreExecutionHook()

	interchurnclassifyautofe = TransformationMain.run(aiforgemltestdata,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Intern", "transformation_label": "String Indexer"}], "feature": "Intern", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "53", "mean": "", "stddev": "", "min": "AMRO ALHAMMEDI", "max": "sofia andom", "missing": "0", "distinct": "53"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern"}, {"transformationsData": [{"feature_label": "InternStartDate", "transformation_label": "String Indexer"}], "feature": "InternStartDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "53", "mean": "", "stddev": "", "min": "1/4/2021", "max": "9/10/2019", "missing": "0", "distinct": "45"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternStartDate"}, {"transformationsData": [{"feature_label": "InternEndDate", "transformation_label": "String Indexer"}], "feature": "InternEndDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "53", "mean": "", "stddev": "", "min": "1/15/2021", "max": "Unkown", "missing": "0", "distinct": "37"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternEndDate"}, {"transformationsData": [{"feature_label": "Intern_MM_PlacementDate", "transformation_label": "String Indexer"}], "feature": "Intern_MM_PlacementDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "53", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "4/17/2024", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_PlacementDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ClassSignupsID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "11178.53", "stddev": "3587.05", "min": "0", "max": "17900", "missing": "0"}, "updatedLabel": "ClassSignupsID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DaysInInternship", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "165.42", "stddev": "149.59", "min": "3", "max": "615", "missing": "0"}, "updatedLabel": "DaysInInternship"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveySent", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.82", "stddev": "1.11", "min": "0.0", "max": "3.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SurveySent"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INTERNSHIP_SCORE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.38", "stddev": "0.18", "min": "0.25", "max": "0.844872556", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INTERNSHIP_SCORE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveysTaken", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.44", "stddev": "0.7", "min": "0.0", "max": "2.333333333", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SurveysTaken"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveysGiven", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.45", "stddev": "0.69", "min": "0.0", "max": "2.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SurveysGiven"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternCancelReasonID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "1.02", "stddev": "1.26", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "InternCancelReasonID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "QuestionsScored", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "5.77", "stddev": "8.88", "min": "0.0", "max": "36.4", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "QuestionsScored"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "WorkEthic", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.22", "stddev": "0.33", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "WorkEthic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Availability", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.23", "stddev": "0.34", "min": "0.0", "max": "0.9933332", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Availability"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TeamPlayer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.23", "stddev": "0.34", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TeamPlayer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "QualityofWork", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.22", "stddev": "0.33", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "QualityofWork"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Trust_2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.22", "stddev": "0.33", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Trust_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Respectful", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.23", "stddev": "0.34", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Respectful"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveyScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.23", "stddev": "0.34", "min": "0.0", "max": "0.9989742", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SurveyScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCamp_FinalScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.19", "stddev": "0.29", "min": "0.0", "max": "0.842105263", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCamp_FinalScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_FinalScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.14", "stddev": "0.21", "min": "0.0", "max": "0.860740578", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_FinalScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCamp_Score", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.19", "stddev": "0.29", "min": "0.0", "max": "0.842105263", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCamp_Score"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Attendance", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.68", "stddev": "1.08", "min": "0.0", "max": "3.368421053", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Attendance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCampCommentScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.68", "stddev": "1.08", "min": "0.0", "max": "3.368421053", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCampCommentScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCampTotalComments", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "1.09", "stddev": "1.91", "min": "0.0", "max": "8.421052632", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCampTotalComments"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Score", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.14", "stddev": "0.21", "min": "0.0", "max": "0.860740578", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Score"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternTechGroupID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "3.89", "stddev": "2.37", "min": "1", "max": "10", "missing": "0"}, "updatedLabel": "InternTechGroupID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Internquit", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "0.45", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Internquit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_WkOrder", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "18.6", "stddev": "34.15", "min": "1.0", "max": "145.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_WkOrder"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Wk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "10.45", "stddev": "13.24", "min": "0.0", "max": "40.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Wk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Yr", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "877.98", "stddev": "1012.32", "min": "0.0", "max": "2024.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Yr"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_HW", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.04", "stddev": "0.22", "min": "0.0", "max": "1.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_HW"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Videos", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.06", "stddev": "0.24", "min": "0.0", "max": "1.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Videos"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_VideosScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.13", "stddev": "0.43", "min": "0.0", "max": "2.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_VideosScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_AutoInt", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.13", "stddev": "0.42", "min": "0.0", "max": "2.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_AutoInt"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Phone", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.02", "stddev": "0.07", "min": "0.0", "max": "0.428571429", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Phone"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_LoginPer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.27", "stddev": "0.34", "min": "0.0", "max": "0.960666667", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_LoginPer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_ActivityScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.34", "stddev": "0.69", "min": "0.0", "max": "3.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_ActivityScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_ParticipationPer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.14", "stddev": "0.26", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_ParticipationPe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_UserComments", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.22", "stddev": "0.44", "min": "0.0", "max": "2.444444444", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_UserComments"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_ActivityScoreChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.14", "stddev": "0.26", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_ActivityScoreCh..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_UserCommentsChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.18", "stddev": "0.31", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_UserCommentsChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_AutoIntChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.07", "stddev": "0.19", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_AutoIntChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_PhoneChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.02", "stddev": "0.05", "min": "0.0", "max": "0.285714286", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_PhoneChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Wk_ActivityScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.07", "stddev": "0.1", "min": "0.0", "max": "0.306190033", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Wk_ActivityScor..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_CC_Answer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.59", "stddev": "2.16", "min": "0.0", "max": "11.82222222", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_CC_Answer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_COE_Participation", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "1.47", "stddev": "2.38", "min": "0.0", "max": "9.466666667", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_COE_Participation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_COE_ProjectUpdates", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.07", "stddev": "0.37", "min": "0.0", "max": "2.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_COE_ProjectUpdates"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Part1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.34", "stddev": "0.69", "min": "0.0", "max": "3.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Part1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Part2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "0.45", "stddev": "1.47", "min": "0.0", "max": "7.622222222", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Part2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Part3", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "1.47", "stddev": "2.23", "min": "0.0", "max": "7.577777778", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Part3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Score_Pre_Total", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "2.27", "stddev": "3.6", "min": "0.0", "max": "15.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Score_Pre_Total"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternLevelGroup", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "2.02", "stddev": "1.08", "min": "0", "max": "4", "missing": "0"}, "updatedLabel": "InternLevelGroup"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Dominance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "24.53", "stddev": "25.55", "min": "0", "max": "89", "missing": "0"}, "updatedLabel": "Dominance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Influencer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "30.64", "stddev": "27.61", "min": "0", "max": "82", "missing": "0"}, "updatedLabel": "Influencer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Steadiness", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "41.04", "stddev": "36.15", "min": "0", "max": "96", "missing": "0"}, "updatedLabel": "Steadiness"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Compliance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "37.43", "stddev": "32.51", "min": "0", "max": "91", "missing": "0"}, "updatedLabel": "Compliance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Theoretical", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "34.66", "stddev": "28.48", "min": "0", "max": "78", "missing": "0"}, "updatedLabel": "Theoretical"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Utilitarian", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "26.58", "stddev": "23.12", "min": "0", "max": "78", "missing": "0"}, "updatedLabel": "Utilitarian"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Aesthetic", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "24.85", "stddev": "21.07", "min": "0", "max": "75", "missing": "0"}, "updatedLabel": "Aesthetic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Social", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "25.91", "stddev": "23.53", "min": "0", "max": "69", "missing": "0"}, "updatedLabel": "Social"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Individualistic", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "26.47", "stddev": "22.69", "min": "0", "max": "70", "missing": "0"}, "updatedLabel": "Individualistic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Traditional", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "23.89", "stddev": "21.31", "min": "0", "max": "73", "missing": "0"}, "updatedLabel": "Traditional"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Flexibility", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "11.05", "stddev": "20.83", "min": "0.0", "max": "81.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Flexibility"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FuturisticThinking", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "4.53", "stddev": "12.51", "min": "0.0", "max": "60.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "FuturisticThinking"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "GoalOrientation", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "12.56", "stddev": "24.96", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "GoalOrientation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InterpersonalSkills", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "13.56", "stddev": "27.5", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "InterpersonalSkills"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Leadership", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "8.83", "stddev": "20.71", "min": "0.0", "max": "83.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Leadership"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Negotiation", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "9.2", "stddev": "21.17", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Negotiation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PersonalResponsibility", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "10.99", "stddev": "20.56", "min": "0.0", "max": "81.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PersonalResponsibility"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Persuasion", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "7.05", "stddev": "18.64", "min": "0.0", "max": "83.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Persuasion"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PlanningOrganizing", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "10.48", "stddev": "20.17", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PlanningOrganizing"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Presenting", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "9.92", "stddev": "22.38", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Presenting"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Teamwork", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "11.0", "stddev": "22.83", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Teamwork"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "WrittenCommunication", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "9.26", "stddev": "22.01", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "WrittenCommunication"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "UnderstandingOthers", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "11.94", "stddev": "23.03", "min": "0.0", "max": "88.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "UnderstandingOthers"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PracticalThinking", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "12.97", "stddev": "24.84", "min": "0.0", "max": "90.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PracticalThinking"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SystemsJudgment", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "11.28", "stddev": "20.77", "min": "0.0", "max": "80.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SystemsJudgment"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SenseOfSelf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "11.19", "stddev": "21.37", "min": "0.0", "max": "80.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SenseOfSelf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RoleAwareness", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "11.26", "stddev": "21.85", "min": "0.0", "max": "88.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RoleAwareness"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SelfDirection", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "53", "mean": "11.11", "stddev": "20.8", "min": "0.0", "max": "76.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SelfDirection"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "UnderstandingOthersBias", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "-0.42", "stddev": "0.53", "min": "-1", "max": "1", "missing": "0"}, "updatedLabel": "UnderstandingOthersBias"}, {"transformationsData": [{"feature_label": "InternCancelReason", "transformation_label": "String Indexer"}], "feature": "InternCancelReason", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "53", "mean": "", "stddev": "", "min": "Not Quit", "max": "Unknown", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternCancelReason"}, {"transformationsData": [{"feature_label": "Phase", "transformation_label": "String Indexer"}], "feature": "Phase", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "53", "mean": "", "stddev": "", "min": "Phase 1", "max": "Unkown", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Phase"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "53", "mean": "", "stddev": "", "min": "F", "max": "Unknown", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "TopDISCscore", "transformation_label": "String Indexer"}], "feature": "TopDISCscore", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "53", "mean": "", "stddev": "", "min": "Compliance", "max": "Unknown", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TopDISCscore"}, {"transformationsData": [{"feature_label": "LowDISCscore", "transformation_label": "String Indexer"}], "feature": "LowDISCscore", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "53", "mean": "", "stddev": "", "min": "Compliance", "max": "Unknown", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LowDISCscore"}, {"transformationsData": [{"feature_label": "TopMotivator", "transformation_label": "String Indexer"}], "feature": "TopMotivator", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "53", "mean": "", "stddev": "", "min": "Aesthetic", "max": "Utilitarian", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TopMotivator"}, {"transformationsData": [{"feature_label": "LowMotivator", "transformation_label": "String Indexer"}], "feature": "LowMotivator", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "53", "mean": "", "stddev": "", "min": "Aesthetic", "max": "Utilitarian", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LowMotivator"}, {"transformationsData": [{"feature_label": "TopSkill", "transformation_label": "String Indexer"}], "feature": "TopSkill", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "53", "mean": "", "stddev": "", "min": "Flexibility", "max": "WrittenCommunication", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TopSkill"}, {"transformationsData": [{"feature_label": "LowSkill", "transformation_label": "String Indexer"}], "feature": "LowSkill", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "53", "mean": "", "stddev": "", "min": "Flexibility", "max": "WrittenCommunication", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LowSkill"}]}))

	#transformationPostExecutionHook(interchurnclassifyautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run interchurnclassify4Hooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(interchurnclassifyautofe, ["ClassSignupsID", "DaysInInternship", "SurveySent", "INTERNSHIP_SCORE", "SurveysTaken", "SurveysGiven", "InternCancelReasonID", "QuestionsScored", "WorkEthic", "Availability", "TeamPlayer", "QualityofWork", "Trust_2", "Respectful", "SurveyScore", "BaseCamp_FinalScore", "IPBC_FinalScore", "BaseCamp_Score", "Attendance", "BaseCampCommentScore", "BaseCampTotalComments", "IPBC_Score", "InternTechGroupID", "Intern_MM_WkOrder", "Intern_MM_Wk", "Intern_MM_Yr", "Intern_MM_HW", "Intern_MM_Videos", "Intern_MM_VideosScore", "Intern_MM_AutoInt", "Intern_MM_Phone", "Intern_MM_LoginPer", "Intern_MM_ActivityScore", "Intern_MM_ParticipationPer", "Intern_MM_UserComments", "Intern_MM_ActivityScoreChk", "Intern_MM_UserCommentsChk", "Intern_MM_AutoIntChk", "Intern_MM_PhoneChk", "Intern_MM_Wk_ActivityScore", "Intern_CC_Answer", "Intern_COE_Participation", "Intern_COE_ProjectUpdates", "IPBC_Part1", "IPBC_Part2", "IPBC_Part3", "IPBC_Score_Pre_Total", "InternLevelGroup", "Dominance", "Influencer", "Steadiness", "Compliance", "Theoretical", "Utilitarian", "Aesthetic", "Social", "Individualistic", "Traditional", "Flexibility", "FuturisticThinking", "GoalOrientation", "InterpersonalSkills", "Leadership", "Negotiation", "PersonalResponsibility", "Persuasion", "PlanningOrganizing", "Presenting", "Teamwork", "WrittenCommunication", "UnderstandingOthers", "PracticalThinking", "SystemsJudgment", "SenseOfSelf", "RoleAwareness", "SelfDirection", "UnderstandingOthersBias", "Intern_stringindexer", "InternStartDate_stringindexer", "InternEndDate_stringindexer", "Intern_MM_PlacementDate_stringindexer", "InternCancelReason_stringindexer", "Phase_stringindexer", "Gender_stringindexer", "TopDISCscore_stringindexer", "LowDISCscore_stringindexer", "TopMotivator_stringindexer", "LowMotivator_stringindexer", "TopSkill_stringindexer", "LowSkill_stringindexer"], "Internquit")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

